# Striker Replacement Project
* Introduction \
Statistics was always a key part of football. Goals, assists and some basic metrics were usually used to evaluate a player’s performance. In recent years, the field of football analytics has developed. Using modern day equipment, more information can be gathered and can be used in decision making. Robert Lewandowski was a prolific striker for Bayern Munich. In his last 3 seasons with the club in 94 matches, scored 110 goals. His goal scoring ability played a key role in the treble winning season 19/20 and 2 Bundesliga championships the following seasons. In the summer of 2022 FC Barcelona spend 45 million euros and the Polish striker moved from Munich to Barcelona. Leaving the German giants without a serial goal scorer which lead to a disastrous Champions League run and an almost losing the domestic title.    

* Project goals \
Bayern Munich is in need of a striker, someone who can fill Lewandowski’s boots and became the new number 9 for the Bauerian giants. The goal is to use data analysis techniques to recommend a list of potential candidates.

* Data source \
Data were retrieved from https://fbref.com/en/

* Required libaries \
pandas, lxml, html5lib, beautifulsoup4, numpy, sklearn, openpyxl

* Data Glossary

| Name | Description |
|------|-------------|
| Nation | Nationality of the player|
| Pos | Position, position most commonly played by the player|
| Comp | Competition. Number next to competition states which level in the country's league pyramid this league occupies.|
| Age | Age at season start|
| Born | Year of birth|
| 90s | 90s Played|
| Gls | Goals|
| Sh | Shots Total. Does not include penalty kicks|
| SoT | Shots on target|
| SoT% | Percentage of shots that are on target|
| Sh/90 | Shots total per 90 minutes|
| SoT/90 | Shots on target per 90 minutes|
| G/Sh | Goals per shot|
| G/SoT | Goals per shot on target|
| Dist | Average distance, in yards, from goal of all shots taken|
| FK | Shots from free kicks|
| PK | Penalty Kicks Made|
| PKatt | Penalty Kicks Attempted|
| xG | Expected Goals, totals include penalty kicks, but do not include penalty shootouts|
| npxG | Non-Penalty Expected Goals|
| npxG/Sh | Non-Penalty Expected Goals per shot|
| G-xG | Goals minus Expected Goals|
| np:G-xG | Non-Penalty Expected Goals-

| Name | Description |
|------|-------------|
| SCA | Shot-Creating Action|
| SCA90 | Shot-Creating Actions per 90 minutes|
| SCA_PassLive | Completed live-ball passes that lead to a shot attempt|
| SCA_PassDead | Completed dead-ball passes that lead to a shot attempt|
| SCA_TO | Successful take-ons that lead to a shot attempt|
| SCA_Sh | Shots that lead to another shot attempt|
| SCA_Fld | Fouls drawn that le
| SCA_Def | Defensive actions that lead to a shot attempt|
| GCA | Goal-Creating Actions|
| GCA90 |Goal-Creating Actions per 90 minutes|
| GCA_PassLive | Completed live-ball passes that lead to a goal|
| GCA_PassDead | Completed dead-ball passes that lead to a goal|
| GCA_TO | Successful take-ons that lead to a goal|
| GCA_Sh | Shots that lead to another goal-scoring shot|
| GCA_Fl | Fouls drawn that lead to a goal|
| GCA_Def| Defensive actiodated when available.  |

In [1]:
!pip install lxml html5lib beautifulsoup4 scikit-learn openpyxl
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn import preprocessing

In [2]:
url = 'https://fbref.com/en/comps/Big5/2022-2023/shooting/players/2022-2023-Big-5-European-Leagues-Stats'
dfs = pd.read_html(url)

In [3]:
Shooting_Stats_2223 = dfs[0]

In [4]:
dfs = pd.read_html('https://fbref.com/en/comps/Big5/2022-2023/gca/players/2022-2023-Big-5-European-Leagues-Stats')

In [5]:
Goal_ShotCreation_Stats_2223 = dfs[0]

In [6]:
Shooting_Stats_2223.columns = ['Rk','Player','Nation','Pos','Squad','Comp','Age','Born','90s','Gls','Sh','SoT','SoT%','Sh/90','SoT/90','G/Sh','G/SoT','Dist','FK','PK','PKatt','xG','npxG','npxG/Sh','G-xG','np:G-xG','Matches']

In [7]:
Shooting_Stats_2223 = Shooting_Stats_2223.loc[Shooting_Stats_2223['Rk'] != 'Rk']
Shooting_Stats_2223 = Shooting_Stats_2223.reset_index(drop = True)

In [8]:
Goal_ShotCreation_Stats_2223.columns = [
    'Rk','Player','Nation','Pos','Squad','Comp','Age','Born','90s','SCA','SCA90','SCA_PassLive','SCA_PassDead','SCA_TO','SCA_Sh','SCA_Fld','SCA_Def','GCA','GCA90','GCA_PassLive','GCA_PassDead'
    ,'GCA_TO','GCA_Sh',	'GCA_Fld','GCA_Def','Matches']

In [9]:
Goal_ShotCreation_Stats_2223 = Goal_ShotCreation_Stats_2223.loc[Goal_ShotCreation_Stats_2223['Rk'] != 'Rk']
Goal_ShotCreation_Stats_2223 = Goal_ShotCreation_Stats_2223.reset_index(drop = True)

In [10]:
Shooting_Leagues_Stats = pd.read_csv("C:/Users/user/Downloads/Shooting_Leagues_Stats.txt",sep='\t')
Goal_ShotCreation_Leagues_Stats = pd.read_csv("C:/Users/user/Downloads/Goal_ShotCreation_Leagues_Stats.txt",sep='\t')

In [11]:
Goal_ShotCreation_Leagues_Stats = Goal_ShotCreation_Leagues_Stats.drop(columns=['MP'])

There is a need to compare the top 5 European football leagues in terms of similarity. Cosine similarity and Euclidian similarity will be used for that because they are the most common one.

In [12]:
#similarity with minmax scaling
numeric_df = Goal_ShotCreation_Leagues_Stats.drop(columns=['Competition_Name'])

x = numeric_df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
numeric_df = pd.DataFrame(x_scaled)


# Calculate cosine similarity
cosine_sim = cosine_similarity(numeric_df)

# Create a new DataFrame to show the similarity matrix
goalshot_similarity_df = pd.DataFrame(cosine_sim, index=Goal_ShotCreation_Leagues_Stats['Competition_Name'],columns=Goal_ShotCreation_Leagues_Stats['Competition_Name'])
print('cosine similarity in goals and shot creation stats of the top 5 European football leagues\n')
print(goalshot_similarity_df)

cosine similarity in goals and shot creation stats of the top 5 European football leagues

Competition_Name  Bundesliga   La_Liga   Ligue_1  Premier_League   Serie_A
Competition_Name                                                          
Bundesliga          1.000000  0.482400  0.615517        0.807887  0.622542
La_Liga             0.482400  1.000000  0.427014        0.601611  0.496859
Ligue_1             0.615517  0.427014  1.000000        0.653911  0.339259
Premier_League      0.807887  0.601611  0.653911        1.000000  0.802752
Serie_A             0.622542  0.496859  0.339259        0.802752  1.000000


euclidean similarity in goals and shot creation stats of the top 5 European football leagues

Competition_Name  Bundesliga   La_Liga   Ligue_1  Premier_League   Serie_A
Competition_Name                                                          
Bundesliga          1.000000  0.259400  0.282184        0.343565  0.284146
La_Liga             0.259400  1.000000  0.336508        0.34232

Generally, the similarity of leagues needs to be a part of any future recommendation. Premier League looks to be the league closer to Bundesliga, that why players from premier league will have a small advantage compare to all the others.  

In [13]:
Shooting_Leagues_Stats = Shooting_Leagues_Stats.drop(columns=['MP'])

In [14]:
#similarity with minmax scaling
numeric_df = Shooting_Leagues_Stats.drop(columns=['Competition_Name'])

x = numeric_df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
numeric_df = pd.DataFrame(x_scaled)


# Calculate cosine similarity
cosine_sim = cosine_similarity(numeric_df)

# Create a new DataFrame to show the similarity matrix
shooting_similarity_df = pd.DataFrame(cosine_sim, index=Shooting_Leagues_Stats['Competition_Name'],columns=Shooting_Leagues_Stats['Competition_Name'])
print('cosine similarity in shooting stats of the top 5 European football leagues\n')
print(shooting_similarity_df)


cosine similarity in shooting stats of the top 5 European football leagues

Competition_Name  Bundesliga   La_Liga   Ligue_1  Premier_League   Serie_A
Competition_Name                                                          
Bundesliga          1.000000  0.509358  0.795336        0.836858  0.429940
La_Liga             0.509358  1.000000  0.689334        0.407755  0.695421
Ligue_1             0.795336  0.689334  1.000000        0.647485  0.260194
Premier_League      0.836858  0.407755  0.647485        1.000000  0.290778
Serie_A             0.429940  0.695421  0.260194        0.290778  1.000000


euclidean similarity in shooting stats of the top 5 European football leagues

Competition_Name  Bundesliga   La_Liga   Ligue_1  Premier_League   Serie_A
Competition_Name                                                          
Bundesliga          1.000000  0.246625  0.317332        0.327384  0.238948
La_Liga             0.246625  1.000000  0.350377        0.323731  0.440746
Ligue_1           

In [15]:
Shooting_Stats_2223 = Shooting_Stats_2223.drop(columns=['Rk','Nation','Born','Matches','Age'])
Shooting_Stats_2223_FW = Shooting_Stats_2223.loc[Shooting_Stats_2223['Pos'] =='FW']
Shooting_Stats_2223_FW['Comp'] = Shooting_Stats_2223_FW['Comp'].str[3:].str.lstrip().str.replace(' ', '_')

C:\Users\user\AppData\Local\Temp\ipykernel_8060\516311213.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Shooting_Stats_2223_FW['Comp'] = Shooting_Stats_2223_FW['Comp'].str[3:].str.lstrip().str.replace(' ', '_')


In [16]:
Shooting_Stats_2223_FW = Shooting_Stats_2223_FW.dropna(axis=0)

In [17]:
data_types_dict = {
    '90s': float,
    'Gls': float,
    'Sh': float,
    'SoT': float,
    'SoT%': float,
    'Sh/90': float,
    'SoT/90': float,
    'G/Sh': float,
    'G/SoT': float,
    'Dist': float,
    'FK': float,
    'PK': float,
    'PKatt': float,
    'xG': float,
    'npxG': float,
    'npxG/Sh': float,
    'G-xG': float,
    'np:G-xG': float
}

In [18]:
Shooting_Stats_2223_FW = Shooting_Stats_2223_FW.astype(data_types_dict)

All the possible cadidates are around 400 to make easier and faster calculations some players will be cut from the list based on their perfomance.

In [19]:
Shooting_Stats_2223_FW =  Shooting_Stats_2223_FW[(Shooting_Stats_2223_FW['Gls']>10) & (Shooting_Stats_2223_FW['90s']>10)]

In [21]:
Goal_ShotCreation_Stats_2223_FW = Goal_ShotCreation_Stats_2223.loc[Goal_ShotCreation_Stats_2223['Pos'] =='FW']
Goal_ShotCreation_Stats_2223_FW['Comp'] = Goal_ShotCreation_Stats_2223['Comp'].str[3:].str.lstrip().str.replace(' ', '_')
Goal_ShotCreation_Stats_2223_FW = Goal_ShotCreation_Stats_2223_FW.dropna(axis=0) 

C:\Users\user\AppData\Local\Temp\ipykernel_8060\1357308068.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Goal_ShotCreation_Stats_2223_FW['Comp'] = Goal_ShotCreation_Stats_2223['Comp'].str[3:].str.lstrip().str.replace(' ', '_')


In [22]:
Goal_ShotCreation_Stats_2223_FW = Goal_ShotCreation_Stats_2223_FW[Goal_ShotCreation_Stats_2223_FW['Player'].isin(Shooting_Stats_2223_FW['Player'])]

In [23]:
player_delete_1 = (Goal_ShotCreation_Stats_2223_FW['Player'] == 'Terem Moffi') & (Goal_ShotCreation_Stats_2223_FW['Squad'] == 'Nice')
player_delete_2 = (Goal_ShotCreation_Stats_2223_FW['Player'] == 'Gaëtan Laborde') & (Goal_ShotCreation_Stats_2223_FW['Squad'] == 'Rennes')
player_delete_3 = (Goal_ShotCreation_Stats_2223_FW['Player'] == 'Alexander Sørloth') & (Goal_ShotCreation_Stats_2223_FW['Squad'] == 'RB Leipzig')

Goal_ShotCreation_Stats_2223_FW = Goal_ShotCreation_Stats_2223_FW[~player_delete_1]
Goal_ShotCreation_Stats_2223_FW = Goal_ShotCreation_Stats_2223_FW[~player_delete_2]
Goal_ShotCreation_Stats_2223_FW = Goal_ShotCreation_Stats_2223_FW[~player_delete_3]

C:\Users\user\AppData\Local\Temp\ipykernel_8060\146984326.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Goal_ShotCreation_Stats_2223_FW = Goal_ShotCreation_Stats_2223_FW[~player_delete_2]
C:\Users\user\AppData\Local\Temp\ipykernel_8060\146984326.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Goal_ShotCreation_Stats_2223_FW = Goal_ShotCreation_Stats_2223_FW[~player_delete_3]


In [24]:
types_dict = {
'90s'  :           float,
'SCA'   :          float,
'SCA90'  :         float,
'SCA_PassLive' :   float,
'SCA_PassDead' :   float,
'SCA_TO'       :   float,
'SCA_Sh'       :  float,
'SCA_Fld'      :   float,
'SCA_Def'      :   float,
'GCA'          :   float,
'GCA90'        :   float,
'GCA_PassLive' :   float,
'GCA_PassDead' :   float,
'GCA_TO'       :   float,
'GCA_Sh'       :   float,
'GCA_Fld'      :  float,
'GCA_Def'      :   float
}

In [25]:
Goal_ShotCreation_Stats_2223_FW =Goal_ShotCreation_Stats_2223_FW.astype(types_dict)

In [26]:
Lewandowski_Shooting_Stats = pd.read_csv("C:/Users/user/Downloads/Robert_Lewandowski_Shooting_Stats.txt",sep='\t')
Lewandowski_Goal_ShotCreation_Stats = pd.read_excel("C:/Users/user/Downloads/Robert_Lewandowski_Goal_Stats.xlsx")

In [56]:
x = Lewandowski_Goal_ShotCreation_Stats.drop(columns=['Player','Pos','Squad','Comp'])
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
x_scaled_mean = x_scaled.mean(axis=0)
Lewandowski_Goal_ShotCreation_vec = x_scaled_mean.transpose()

In [57]:
y = Goal_ShotCreation_Stats_2223_FW.drop(columns=['Rk','Nation','Player','Age','Born','Pos','Squad','Comp','Matches'])
y = y.values
y_scaled = min_max_scaler.fit_transform(y)
Goal_ShotCreation_array = y_scaled

In [68]:
cosine_GSC = cosine_similarity(Goal_ShotCreation_array,Lewandowski_Goal_ShotCreation_vec.reshape(1, -1))
cosine_GSC_df = pd.DataFrame(cosine_GSC, index=Goal_ShotCreation_Stats_2223_FW['Player'])
cosine_GSC_df.columns = ['Cosine GSC Similarity']
cosine_GSC_df.sort_values(by=['Cosine GSC Similarity'],ascending=False)

,Cosine GSC Similarity
Player,
Harry Kane,0.973655
Khvicha Kvaratskhelia,0.952693
Mama Samba Baldé,0.942678
Folarin Balogun,0.936538
Alexandre Lacazette,0.926722
Jonathan David,0.921616
Gabriel Jesus,0.919210
Victor Osimhen,0.918889
Marcus Rashford,0.913600


In [79]:
x = Lewandowski_Shooting_Stats.drop(columns=['Player','Pos','Squad','Comp'])
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
x_scaled_mean = x_scaled.mean(axis=0)
Lewandowski_Goal_Shooting_Stats_vec = x_scaled_mean.transpose()

In [75]:
y = Shooting_Stats_2223_FW.drop(columns=['Player','Pos','Squad','Comp'])
y = y.values
y_scaled = min_max_scaler.fit_transform(y)
Shooting_Stats_array = y_scaled

In [81]:
cosine_ST = cosine_similarity(Shooting_Stats_array,Lewandowski_Goal_Shooting_Stats_vec.reshape(1, -1))
cosine_ST_df = pd.DataFrame(cosine_ST, index=Goal_ShotCreation_Stats_2223_FW['Player'])
cosine_ST_df.columns = ['Cosine ST Similarity']
cosine_ST_df.sort_values(by=['Cosine ST Similarity'],ascending=False)

,Cosine ST Similarity
Player,
Alexandre Lacazette,0.959859
Olivier Giroud,0.958398
Marcus Rashford,0.937704
Wissam Ben Yedder,0.937673
Ivan Toney,0.937072
Habib Diallo,0.936407
Loïs Openda,0.927882
Erling Haaland,0.918994
Callum Wilson,0.918049


There are many options which stand out. But there are some factors that can be crucial in decision making. That is why in these project leagues similarities were also calculated. It is also obvious that the new player must be similar to Lewandowski in both goal/shot creation and shooting. Below there a table with the players who have similarity above 0.90 in both.


|Player|Goal Shot Creation Similarity|Shooting Similarity|
|------|-----------------------------|-------------------|
|Harry Kane|	0.973655| 0.913772|
|Alexandre Lacazette|	0.926722|0.959859|
|Victor Osimhen	 |0.918889	|0.905357|
|Marcus Rashford|0.913600|0.937704|
|Kylian Mbappé	|0.911079|0.911028|
|Randal Kolo Muani	|0.904074|0.915445|



On August 12 Bayern paid Spurs one hundred million euros for Harry Kane, and the Englishman became the new number 9 for the Bavarian club. After the previous data analysis, the move is justified.